In [ ]:
import json
import cv2
import os
import numpy as np
import torch
import torchvision
import pycocotools
import cv2 as cv
from torchvision import transforms
from torchvision.models.detection.rpn import AnchorGenerator

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Device type:    {device}")

In [ ]:
def get_keypoints_model():
    anchor_generator = AnchorGenerator(sizes=(32, 64, 128, 256, 512), aspect_ratios=(0.25, 0.5, 0.75, 1.0, 2.0, 3.0, 4.0))
    model=torchvision.models.detection.keypointrcnn_resnet50_fpn(pretrained=False,pretrained_backbone=True,num_keypoints=1,num_classes=7,rpn_anchor_generator=anchor_generator)
    return model

In [ ]:
model = get_keypoints_model()

In [ ]:
checkpoint=torch.load("outputs\checkpoint_epoch30.pth")
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

In [ ]:
model = torch.load("outputs\kp_model.pth")
model.eval()

In [ ]:
with open("data\img_test_ann.json") as coco_file:
  coco = json.load(coco_file)

In [ ]:
model = model.to(device)

In [ ]:
global_path = "data\\test_images"
font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,30)
fontScale              = 1
fontColor              = (255,255,255)
thickness              = 1
lineType               = 2
categ={1:'oats',2:'tea',3:'crisps',4:'soup',5:'scissors',6:'pan'}
for item in coco["images"]:
  file_name = item["file_name"]
  img_path = os.path.join(global_path,file_name)
  img = cv.imread(img_path)
  imgTens=torchvision.transforms.functional.to_tensor(img)
  imgTens = imgTens.to(device)
  with torch.no_grad():
    output = model([imgTens])
  for idx in range(0,6):
    bbox=output[0]['boxes'][idx]
    cat=output[0]['labels'][idx]
    p0 =output[0]['keypoints'][idx][0]
    img= cv2.rectangle(img,(int(bbox[0].item()),int(bbox[1].item())),(int(bbox[2].item()),int(bbox[3].item())),(255,0,255),3)
    img= cv2.circle(img,(int(p0[0]),int(p0[1])),4,(255,255,255),5) #grasping point
    cv2.putText(img,categ[int(cat.item())],(int(bbox[0].item())+40,int(bbox[1].item())) , font, fontScale,fontColor,thickness,lineType)
  cv2.imwrite(os.path.join('tested_net',file_name),img)
                